## Regression modeling
---
- Linear Regression
- Decision Tree
- Random Forest
- SVR
---
- Database: Postgre SQL

In [1]:
import pandas as pd
import numpy as np
import psycopg2 as pg2

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error

### Fetch data from database

In [2]:
def get_cur():
    db_pwd = 'password'
    conn = pg2.connect(database='AQI_2019', user='postgres', password=db_pwd)
    return conn.cursor()

def close():
    cur.close()
    conn.close()

cur = get_cur()

In [3]:
# fetch data
cur.execute("""SELECT * FROM aqi
            INNER JOIN measuring_pt ON measuring_pt.pt_id = aqi.pt_id
            WHERE aqi.type='PM25'
            """)
df = pd.DataFrame(cur.fetchall(), columns=['date', 'type', 'state', 'aqi', 'id1', 'lat', 'long', 'pt_id']).drop(columns=['id1', 'type'])

In [4]:
df.head()

,date,state,aqi,lat,long,pt_id
0,2019-06-08,Louisiana,40.0,30.041238,-90.272826,121
1,2019-06-14,Louisiana,35.0,30.041238,-90.272826,121
2,2019-06-20,Louisiana,51.0,30.041238,-90.272826,121
3,2019-06-26,Louisiana,59.0,30.041238,-90.272826,121
4,2019-07-02,Louisiana,19.0,30.041238,-90.272826,121


In [5]:
df.shape

(213149, 6)

### Data preparation for modeling

In [8]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = pd.DatetimeIndex(df['date']).month.map(str) + pd.DatetimeIndex(df['date']).day.map(str)
df['date'] = pd.to_datetime(df['date'], format='%m%d').dt.strftime('%m%d')

In [10]:
df['date'] = df['date'].map(lambda x: int(x))

In [11]:
X = df.drop(columns=['aqi', 'pt_id'])
y = df['aqi']

In [12]:
X.head()

,date,state,lat,long
0,608,Louisiana,30.041238,-90.272826
1,614,Louisiana,30.041238,-90.272826
2,620,Louisiana,30.041238,-90.272826
3,626,Louisiana,30.041238,-90.272826
4,702,Louisiana,30.041238,-90.272826


In [13]:
# Control column by column when onehotencoding
# but when using 'passthrough', we can't get the feature names

# ct = ColumnTransformer([('ohenc', OneHotEncoder(drop='first'), [1])], remainder='passthrough')
# X = np.array(ct.fit_transform(X))

In [14]:
# We use get_dummies because it makes exogenous(X) readable
X = pd.get_dummies(X, columns=['state'], drop_first=True)
X.head()

,date,lat,long,state_Alaska,state_Arizona,state_Arkansas,state_California,state_Colorado,state_Connecticut,state_Delaware,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,608,30.041238,-90.272826,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,614,30.041238,-90.272826,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,620,30.041238,-90.272826,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,626,30.041238,-90.272826,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,702,30.041238,-90.272826,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
# Prepare data for Linear Regression
X_sm = df[df['state'] == 'Alabama'].drop(columns=['aqi', 'pt_id', 'state'])
y_sm = df[df['state'] == 'Alabama']['aqi']
X_sm.shape, y_sm.shape

((2065, 3), (2065,))

In [17]:
X_sm_train, X_sm_test, y_sm_train, y_sm_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=42)

### Linear Regression

In [39]:
lr = LinearRegression(n_jobs=2)

In [40]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=2, normalize=False)

In [41]:
lr.score(X_train, y_train), lr.score(X_test, y_test)

(0.12137668228485576, 0.1184871647140716)

In [42]:
lr.coef_

array([ 2.93547336e-03, -3.48538171e-01,  3.11704142e-01,  2.34737212e+01,
       -7.19921314e-01,  2.83277615e+00,  5.12221525e+00,  2.42902056e-01,
       -8.22953018e+00, -6.87871666e+00, -3.43990448e+00, -8.73088865e+00,
        8.85938653e-01, -6.42880211e+00, -5.61133940e+00,  3.86235883e+00,
        1.97658848e+00,  1.11642893e+00, -9.54500167e-04,  1.11342816e+00,
       -1.22492305e+00, -1.26336536e+01, -8.10534898e+00, -1.01725690e+01,
        2.52474710e+00, -3.63114816e+00,  3.61104556e-01,  1.98978453e-01,
        1.70810091e-01, -2.21908827e+00, -2.50066672e+00, -1.80438340e+01,
       -4.85635212e+00, -3.76474851e+00, -8.27527352e+00, -5.15865426e+00,
       -6.37804768e+00,  1.59832719e+00,  3.16032757e+00,  1.08651435e+01,
       -4.11517261e-01, -1.50532717e+01, -6.06625670e+00, -8.77769073e+00,
       -4.37938913e+00,  2.11007510e+00, -1.53251368e+00, -1.14140715e+01,
       -7.00023110e+00,  7.68847454e+00, -2.57501522e+00, -7.79287499e-01,
       -1.26742528e+01])

In [43]:
lr.intercept_

72.09596906722338

In [44]:
y_pred = lr.predict(X_test)

In [45]:
mean_squared_error(y_test, y_pred)

236.38999429262887

#### Assumptions of SLR(Simple Linear Regression) & MLR (Multiple Linear Regression)

---

1 ~ 4 for SLR, 1 ~ 5 for MLR

1. **Linearity:** $Y$ must have an approximately linear relationship with each $X$ variable.
2. **Independence of Errors:** Errors (residuals) $\varepsilon_i$ and $\varepsilon_j$ must be independent of one another for any $i \ne j$.
3. **Normality:** The errors (residuals) follow a Normal distribution with mean 0.
4. **Equality of Variances**: The errors (residuals) should have a roughly consistent pattern, regardless of the value of the $X$ variables. (There should be no discernable relationship between the $X$ variable and the residuals.)
5. **Independence of Predictors** (_almost always violated at least a little!_): The independent variables $X_i$ and $X_j$ must be independent of one another for any $i \ne j$.

The mnemonic LINEI is a useful way to remember these five assumptions.

### Decision Tree Regressor

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [106]:
dtr = DecisionTreeRegressor(max_depth=5,
                            min_samples_split=7,
                            min_samples_leaf=3,
                            random_state=42)

In [112]:
dtr.fit(X_train, y_train)
dtr.score(X_train, y_train), dtr.score(X_test, y_test)

(0.1439059907931195, 0.14036969126290377)

#### Grid Search with Decision tree

In [123]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=42),
                    param_grid={
                        'max_depth': [20, 30],
                        'min_samples_split': [2, 5, 10],
                        'min_samples_leaf': [2, 5, 6, 7]
                    },
                    cv = 5)

In [124]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=42,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [20, 30],
                         'min_samples_leaf': [2, 5, 6, 7],
                         'min_samples_split': [2, 5, 10]},
             pre_dispatch='2*n_jobs', refit=True

In [126]:
gs.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=30, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=42, splitter='best')

In [128]:
gs.best_score_

0.5960969323197362

In [127]:
gs_best = gs.best_estimator_
gs_best.fit(X_train, y_train)
gs_best.score(X_train, y_train), gs_best.score(X_test, y_test)

(0.8101973678310828, 0.6316494050747716)

In [134]:
preds = gs_best.predict(X_test)

### Random Forest Regressor

In [132]:
rfr = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=4, random_state=42)
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [133]:
rfr.score(X_train, y_train), rfr.score(X_test, y_test)

(0.8558787633479172, 0.699795503891673)

#### Grid search

In [136]:
gs_rfr = GridSearchCV(estimator=RandomForestRegressor(random_state=42, n_jobs=4),
                    param_grid={
                        'n_estimators': [100, 200],
                        'max_depth': [20, 30]
                    },
                    cv = 5)
gs_rfr.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=4,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [20, 30], 'n_estimators

In [137]:
gs_rfr.best_params_

{'max_depth': 30, 'n_estimators': 200}

In [138]:
gs_rfr.best_score_

0.705215057584567

In [139]:
gs_rfr_best = gs_rfr.best_estimator_
gs_rfr_best.fit(X_train, y_train)
gs_rfr_best.score(X_train, y_train), gs_rfr_best.score(X_test, y_test)

(0.9450002592474513, 0.7274030239933473)

### SVR

In [38]:
svr = SVR(C=22, epsilon=1) # Larger C leads to the overfitting, smaller C to the underfitting
svr.fit(X_sm_train, y_sm_train)
svr.score(X_sm_train, y_sm_train), svr.score(X_sm_test, y_sm_test)

C:\Users\jjing\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


(0.8752388813368439, 0.6656864548413307)